# Exploratory Data Analysis & Data Cleanup

In [1]:
# Imported Modules/Packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

In [2]:
# Load the dataset
pd.set_option('display.max_columns', None)

X = pd.read_csv(r'../data/Curated_Patent_Drug_Dataset.csv').drop(columns=['Compound_CID', 'SMILE', 'name', 'source', 
                                                                       'IC50 (uM)', 'SMILES_canon', 'Morgan_FP', 
                                                                       'MACCS_FP']) # uses relative path
                                                                                    # removes descriptors that are not numeric

y = X['pIC50'] if 'pIC50' in X.columns else None # this is our predicted value
X.head(5)

pIC50  Molecular_Weight Molecular_Formula  Polar_Area  Complexity  \
0  5.507240            249.40        C12H11NOS2        86.5         330   
1  5.576918            488.40        C23H25IN2S        35.4         567   
2  5.244163            285.23       C11H5F2NO4S        89.9         468   
3  7.397940            924.10        C47H73NO17       320.0        1670   
4  5.020452            678.50      C20H34AuO9PS       115.0         532   

   xlogp  heavycnt  hbonddonor  hbondacc  Rotatable_Bonds   exactmass  \
0    3.6        16           1         3                2  249.028214   
1    NaN        27           0         3                3  488.078308   
2    2.8        19           1         7                1  284.990723   
3    0.0        65          12        18                3  923.487854   
4    NaN        32           0        10               12  678.132690   

   monoisotopicmass  charge  covalentunitcnt  isotopeatomcnt  \
0        249.028214       0                1               0   
1        488.078308       0                2               0   
2        284.990723       0                1               0   
3        923.487854       0                1               0   
4        678.132690       0                3               0   

   totalatomstereocnt  definedatomstereocnt  undefinedatomstereocnt  \
0                   0                     0                       0   
1                   0                     0                       0   
2                   0                     0                       0   
3                  19                    19                       0   
4                   5                     5                       0   

   totalbondstereocnt  definedbondstereocnt  undefinedbondstereocnt  nAcid  \
0                   1                     1                       0      0   
1                   2                     2                       0      0   
2                   1                     1                       0      0   
3                   7                     7                       0      1   
4                   0                     0                       0      0   

    ALogP     ALogp2       AMR  nA  nR  nN  nD  nC  nF  nQ  nE  nG  nH  nI  \
0  2.5899   6.707582   76.6809   0   0   0   0   0   0   0   0   0   0   0   
1  1.8576   3.450678  121.8796   0   0   0   0   0   0   0   0   0   0   0   
2  2.4175   5.844306   66.7596   0   0   0   0   0   0   0   0   0   0   0   
3 -5.1814  26.846906  230.9353   0   0   0   0   0   0   0   0   0   0   0   
4 -0.5367   0.288047   69.9648   0   0   0   0   0   0   0   0   0   0   0   

   nP  nL  nK  nM  nS  nT  nY  nV  nW        apol  naAromAtom  nAromBond  \
0   0   0   0   0   0   0   0   0   0   36.156723           6          6   
1   0   0   0   0   0   0   0   0   0   62.249825          15         16   
2   0   0   0   0   0   0   0   0   0   31.015965           6          6   
3   0   0   0   0   0   0   0   0   0  146.129889           0          0   
4   0   0   0   0   0   0   0   0   0   47.427067           0          0   

   nAtom  nHeavyAtom  nH.1  nB  nC.1  nN.1  nO  nS.1  nP.1  nF.1  nCl  nBr  \
0     27          16    11   0    12     1   1     2     0     0    0    0   
1     51          26    25   0    23     2   0     1     0     0    0    0   
2     24          19     5   0    11     1   4     1     0     2    0    0   
3    138          65    73   0    47     1  17     0     0     0    0    0   
4     43          24    19   0    14     0   9     1     0     0    0    0   

   nI.1  nX         ATS0m         ATS1m         ATS2m         ATS3m  \
0     0   0   4250.197406   3405.953390   5678.678562   4495.813516   
1     0   0   4763.712081   5111.797523   7905.712783   8574.221018   
2     0   2   4561.775952   4076.909547   7282.889398   5378.657251   
3     0   0  11402.238425  11627.627595  17257.990916  20335.877231   
4     0   0   5370.558519   4603.780267   7116.078775   7637.249951   

          ATS

In [3]:
# Show the shape of the dataframe
print(f'Dataframe shape: {X.shape}') # m=112 n=1573
print('=============================')

# Note: 
# This is an incredibly large dataset where there are n > m predictors.
# This affects numerical stability and risks for overfitting and reduced 
# model performance. We proceed to numerically the dataset to go from 
# 1573 predictors to some around 500. 
#
# 12/24 - New predictor count: 1563 

print(X.dtypes)

Dataframe shape: (112, 1565)
pIC50                float64
Molecular_Weight     float64
Molecular_Formula     object
Polar_Area           float64
Complexity             int64
                      ...   
WTPT-5               float64
WPATH                  int64
WPOL                   int64
XLogP                float64
Zagreb                 int64
Length: 1565, dtype: object


In [4]:
# We clean the dataset to make sure that we remove
# all the values or columns that are not workable. Columns
# that are not of numeric type will be removed from the updated
# dataframe; from this we'll be able to work with it.

NaN_Threshold = 0.25 # accepted NaN threshold value

X_not_numeric = X.select_dtypes(include='number') # drops columns that aren't numeric

# We'll measure how much of the dataset is 'NaN'
X_not_numeric.isna().mean().sort_values(ascending=False) # checks how many columns have mean() NaN
X_not_NaN = X_not_numeric.loc[:, X_not_numeric.isna().mean() <= NaN_Threshold] # drops all columns if NaN > threshold
print(f'Matrix shape: {X_not_NaN.shape}')

X_not_NaN.head()


Matrix shape: (112, 1559)


Molecular_Weight  Polar_Area  Complexity  heavycnt  hbonddonor  hbondacc  \
0            249.40        86.5         330        16           1         3   
1            488.40        35.4         567        27           0         3   
2            285.23        89.9         468        19           1         7   
3            924.10       320.0        1670        65          12        18   
4            678.50       115.0         532        32           0        10   

   Rotatable_Bonds   exactmass  monoisotopicmass  charge  covalentunitcnt  \
0                2  249.028214        249.028214       0                1   
1                3  488.078308        488.078308       0                2   
2                1  284.990723        284.990723       0                1   
3                3  923.487854        923.487854       0                1   
4               12  678.132690        678.132690       0                3   

   isotopeatomcnt  totalatomstereocnt  definedatomstereocnt  \
0               0                   0                     0   
1               0                   0                     0   
2               0                   0                     0   
3               0                  19                    19   
4               0                   5                     5   

   undefinedatomstereocnt  totalbondstereocnt  definedbondstereocnt  \
0                       0                   1                     1   
1                       0                   2                     2   
2                       0                   1                     1   
3                       0                   7                     7   
4                       0                   0                     0   

   undefinedbondstereocnt  nAcid   ALogP     ALogp2       AMR  nA  nR  nN  nD  \
0                       0      0  2.5899   6.707582   76.6809   0   0   0   0   
1                       0      0  1.8576   3.450678  121.8796   0   0   0   0   
2                       0      0  2.4175   5.844306   66.7596   0   0   0   0   
3                       0      1 -5.1814  26.846906  230.9353   0   0   0   0   
4                       0      0 -0.5367   0.288047   69.9648   0   0   0   0   

   nC  nF  nQ  nE  nG  nH  nI  nP  nL  nK  nM  nS  nT  nY  nV  nW        apol  \
0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   36.156723   
1   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   62.249825   
2   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   31.015965   
3   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0  146.129889   
4   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   47.427067   

   naAromAtom  nAromBond  nAtom  nHeavyAtom  nH.1  nB  nC.1  nN.1  nO  nS.1  \
0           6          6     27          16    11   0    12     1   1     2   
1          15         16     51          26    25   0    23     2   0     1   
2           6          6     24          19     5   0    11     1   4     1   
3           0          0    138          65    73   0    47     1  17     0   
4           0          0     43          24    19   0    14     0   9     1   

   nP.1  nF.1  nCl  nBr  nI.1  nX         ATS0m         ATS1m         ATS2m  \
0     0     0    0    0     0   0   4250.197406   3405.953390   5678.678562   
1     0     0    0    0     0   0   4763.712081   5111.797523   7905.712783   
2     0     2    0    0     0   2   4561.775952   4076.909547   7282.889398   
3     0     0    0    0     0   0  11402.238425  11627.627595  17257.990916   
4     0     0    0    0     0   0   5370.558519   4603.780267   7116.078775   

          ATS3m         ATS4m         ATS5m         ATS6m         ATS7m  \
0   4495.813516   3680.083923   3191.774767   2873.849213   2198.672601   
1   8574.221018   7649.461376   5467.541446   4607.944744   5465.811944   
2   5378.657251   4449.003866   4111.091646   3640.775691   3391.186705   
3  20335.877231  22277.505624  2012

#### Analysis: 
- Based off our analysis of this dataset there is a lot of multicollinearity and multiple predictors of the same family. We proceed to go to [Analysis 2](exploratory_analysis_2.ipynb) for further analysis in our compressed dataset.